In [1]:
%load_ext autoreload
%autoreload 2

from solution import matrix_to_sycamore_operations

import cirq
from cirq.circuits import Circuit
import numpy as np
from scipy.stats import unitary_group
from cirq.sim import Simulator
sim = Simulator()

In [2]:
# Example
num_qubits = 4
qubits =[cirq.GridQubit(0,i) for i in range(num_qubits)]

A = unitary_group.rvs(2**num_qubits)

In [3]:
# Example
ops, _ = matrix_to_sycamore_operations(qubits, A)

circuit = Circuit(ops)
print(circuit.to_text_diagram(transpose=True))

(0, 0)                            (0, 1)                          (0, 2)                           (0, 3)
│                                 │                               │                                │
@─────────────────────────────────@───────────────────────────────@────────────────────────────────PhXZ(a=-0.00341,x=0.431,z=-0.283)
│                                 │                               │                                │
@─────────────────────────────────@───────────────────────────────(0)──────────────────────────────X
│                                 │                               │                                │
@─────────────────────────────────@───────────────────────────────PhXZ(a=-0.988,x=0.833,z=0.977)───(0)
│                                 │                               │                                │
@─────────────────────────────────@───────────────────────────────(0)──────────────────────────────X
│                                 │                 

In [4]:
def simplify_fc_op(fc_op):

    op = fc_op
    sub_gate = op.gate.sub_gate
    matrix = cirq.unitary(sub_gate)

    control_values = op.gate.control_values
    ordered_qubits = list(op.qubits)
    control_qubits, target_qubit = ordered_qubits[:-1], ordered_qubits[-1]

    # Check if control is 0 valued. If so, add an X gate before and after
    x_gates = []
    for i, val in enumerate(control_values):
        if val == (0,):
            x = cirq.X(control_qubits[i])
            x_gates.append(x)

    ops_decomp = cirq.decompose_multi_controlled_rotation(matrix=matrix, 
                                                          controls=control_qubits, 
                                                          target=target_qubit, )

    # Reverse x gates
    ops_decomp_xs = []
    ops_decomp_xs.extend(x_gates)
    ops_decomp_xs.extend(ops_decomp)
    ops_decomp_xs.extend(x_gates)

    return ops_decomp_xs


In [5]:
simplified_ops = []
for op in ops:
    simplified_ops += simplify_fc_op(op)
    

In [7]:
len(simplified_ops)

10785

In [ ]:
simplified_ops

In [9]:
circ = Circuit(simplified_ops)

In [10]:
circ

KeyboardInterrupt: 